# Manipolazione e classificazione di immagini con OpenCV

**OpenCV** è la libreria di riferimento per l'analisi e l'elaborazione di immagini in tempo reale in Python.\ 
Installiamola nel nostro ambiente JupyterLite

In [ ]:
%pip install opencv-python

**numpy** è un modulo fondamentale per elaborare lunghe serie di numeri organizzati in liste e strutture N-dimensionali.\
Installiamolo

In [ ]:
%pip install numpy

Adesso possiamo importare i moduli

In [ ]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from glob import glob

  Il metodo `glob` crea una **lista** con il nome dei files di una directory

In [ ]:
dog_files = glob('training/dogs/*.jpg')
cat_files = glob('training/cats/*.jpg')

## Visualizzazione delle immagini

Sappiamo già che le immagini sono fatte di pixel. Il pixel è una unità di informazione che contiene i dati necessari a visualizzare un punto dell'immagine.

Vediamo come OpenCV rappresenta un immagine. Prendiamo l'elemento 20 della lista dei files dei "cats".

In [ ]:
cat = cv2.imread(cat_files[20])

In [ ]:
cat

OpenCV rappresenta l'immagine come un 3-array, cioè un array a 3 dimensioni di numpy.

L'attributo `shape` dell' N-array di `numpy` ci mostra la struttura di questo 3-array

In [ ]:
cat.shape

Sostanzialmente lo possiamo vedere come una lista-di-liste-di-liste, in cui 
- La lista di primo livello contiene le **righe** dell'immagine come liste
- ciascuna lista di riga contiene la **lista dei pixel** della riga
- Ciascun elemento della riga è una **lista a 3 valori**: contiene i valori di intensità dei tre colori fondamentali di cascun pixel nella forma **BGR** (blue, green, red). Le intensità sono rappresnetate da un BYTE, per cui il valore è un **intero tra 0 e 255**

Perciò questa immagine ha 270 righe e 286 pixel per riga (270 x 286)

Utilizziamo MatplotLib per visualizzare l'immagine.\
Siccome Matplotlib usa lo standard RGB, dovremo convertire da BGR ad RGB, con un metodo di conversione

In [ ]:
cat_rgb = cv2.cvtColor(cat, cv2.COLOR_BGR2RGB)
fig, ax = plt.subplots(figsize=(6,6))
ax.imshow(cat_rgb)
plt.show()

## I canali RGB

Immaginiamo di estrarre dal 3-array dell'immagine, TRE 2-array che contiene, ciascuno, uno dei 3 elementi più interni del 3-array.\
Questi tre 2-array rappresentano le intensità di ciascun colore BLUE, GREED e RED immaginando che nell'immagine ci sia SOLO quel colore.

Queste stre strutture rappresentano i **canali colore** dell'immagine, che possiamo visualizzare singolarmente

In [ ]:
fig, ax2 = plt.subplots(1,3,figsize=(15,5))
ax2[0].imshow(cat[:,:,0], cmap='Blues')
ax2[1].imshow(cat[:,:,1], cmap='Greens')
ax2[2].imshow(cat[:,:,2], cmap='Reds')
plt.show()

## Processamento delle immagini

### Conversione in scala di grigio

La riduzione dei dati rappresentativi di un immagine è un operazione frequente, quando dell'immagine ci interessanpo più le forme che i colori.

Vediamo questa nuova immagine

In [ ]:
dog = cv2.imread(dog_files[26])
dog_rgb = cv2.cvtColor(dog, cv2.COLOR_BGR2RGB)
fig, ax = plt.subplots(figsize=(6,6))
ax.imshow(dog_rgb)
plt.show()

Una di queste operazioni è la **conversione in scala di grigi**

In [ ]:
dog_gray = cv2.cvtColor(dog, cv2.COLOR_BGR2GRAY)
fig, ax = plt.subplots(figsize=(6,6))
ax.imshow(dog_gray[:,:], cmap='Greys')
plt.show()

Vediamo come è rappresentata questa immagine da OpenCV

In [ ]:
dog_gray

In [ ]:
dog_gray.shape

Vediamo che abbiamo un 2-array. Cosa rappresenta?

Questa conversione ha prodotto un'immagine negativa, perchè l'intensità del grigio è minima nei punti bianchi.

Se ricordiamo che i valori dei pixel sono BYTES che vanno da 0 a 255, come possiamo ripottarla al positivo?

Possiamo sostituire ad ogni valore il risultato di (255 - quel valore)

In [ ]:
all_white = np.full((488, 499), 255)
dog_gray_pos = all_white - dog_gray
fig, ax = plt.subplots(figsize=(6,6))
ax.imshow(dog_gray_pos[:,:], cmap='Greys')
plt.show()

### Ridimensionamento

Anche il ridimensionamento è una operazione frequente, as esempio, quando dobbiamo elaborare una immagine con un programma che si aspetta una certa dimensione massima in pixel delle immagini.

In [ ]:
dog_resized = cv2.resize(dog_rgb, None, fx=0.25, fy=0.25)
fig, ax = plt.subplots(figsize=(6,6))
ax.imshow(dog_resized[:,:])
plt.show()

Questa prima chiamata al metodo `.resize()` opera un ridimensionamento proporzionale su X e Y.\
L'immagine si è sgranata (si dice "pixelata") perchè un minor numero di pixel è stato espanso sulla stessa dimensione in pollici del subplot.

In [ ]:
dog_resized.shape

Vediamo ora una scalatura diversa, che ridimensiona l'immagine su dimensioni prefissate su righe e colonne

In [ ]:
dog_resized = cv2.resize(dog_rgb, (100,200))
fig, ax = plt.subplots(figsize=(6,6))
ax.imshow(dog_resized[:,:])
plt.show()

In [ ]:
dog_resized.shape

# Rilevamento di oggetti o persone

Il rilevamento di oggetti e persone è una delle applicazioni piu frequenti di algoritmi AI che utilizzano reti neurali.\

Il programma della prossima cella rileva il volto di una persona davanti alla webcam e circonda con un quadrato sia il volto che gli occhi.\
O meglio, quello che l'algoritmo ritiene somigliare ad occhi.

Per prima cosa scarichiamo i due **modelli pre-addestrati** dei link seguenti (File -> Open from URL...)
- https://tech.agilioty.com/Day13/haarcascade_frontalface_default.xml
- https://tech.agilioty.com/Day13/haarcascade_eye.xml

Ora, analizziamo il programma nel dettaglio, per capire l'approccio, e poi proviamolo.

In [ ]:
import numpy as np
import cv2

# sorgente dei modelli: https://github.com/Itseez/opencv/tree/master/data/haarcascades

# Caricamento dei modelli pre-addestrati:
# https://github.com/Itseez/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
# https://github.com/Itseez/opencv/blob/master/data/haarcascades/haarcascade_eye.xml
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

cap = cv2.VideoCapture(0)

while 1:
    ret, img = cap.read()
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_img, 1.3, 5)

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        cut_gray = gray_img[y:y+h, x:x+w]
        cut_color = img[y:y+h, x:x+w]
        
        eyes = eye_cascade.detectMultiScale(cut_gray)
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(cut_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

    cv2.imshow('img',img)
    k = cv2.waitKey(30) & 0xff
# 27 = ESC
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()